Specific game:
- https://api.rawg.io/api/games/rimworld
- https://api.rawg.io/api/games/grand-theft-auto-v
- https://rawg.io/games/grand-theft-auto-v

## Get info of each game

In [1]:
import csv
import requests
import json
from pprint import pprint
from time import time
import concurrent.futures
import functools
import os

In [2]:
csv_data = []
with open("data/game_id.csv", "r") as f:
    csv_data = list(csv.reader(f))
for i, val in enumerate(csv_data):
    print(val)
    if i==10: break

['3498', 'grand-theft-auto-v']
['4200', 'portal-2']
['3328', 'the-witcher-3-wild-hunt']
['12020', 'left-4-dead-2']
['5679', 'the-elder-scrolls-v-skyrim']
['802', 'borderlands-2']
['13536', 'portal']
['3439', 'life-is-strange-episode-1-2']
['4291', 'counter-strike-global-offensive']
['4062', 'bioshock-infinite']
['1030', 'limbo']


In [3]:
def threading(start_index, games_per_worker, urls, downloaded_files):
    for url in urls[start_index : start_index + games_per_worker]:
        if url.rsplit("/")[-1] in downloaded_files: continue
#         try:
        # Get json reqquest
        json_data = json.loads(requests.get(url, headers=headers).text)

        # Only include wanted keys
        D = {k:v for k,v in json_data.items() if k in include}

        # Clean up dictionary
        D["platforms"] = []
        for platform in json_data["platforms"]:
            D["platforms"].append(platform["platform"]["name"])

        for key in ("developers", "genres", "publishers"):
            D[key] = []
            for data in json_data[key]:
                D[key].append(data["name"])

        if json_data["esrb_rating"]:
            D["esrb_rating"] = json_data["esrb_rating"]["name"]

        name = D["id"]
        with open(f"data/game_info/{name}.json","w", encoding="utf-8") as f:
            json.dump(D, f)
#         except:
#             print(f"Error with {url}")

## Multithreading

### Create folder if not existed

In [4]:
import os
if not os.path.exists('data/game_info/'):
    os.makedirs('data/game_info/')

#### Preparation

In [5]:
headers = { 'User-Agent': 'App Name: Education purpose',}
include = {"id",
           "slug",
           "name",
           "metacritic",
           "released",
           "tba",
           "updated",
           "website",
           "rating",
           "rating_top",
           "added_by_status",
           "playtime",
           "achievements_count",
           "ratings_count",
           "suggestions_count",
           "game_series_count",
           "reviews_count",
           "platforms",
           "developers",
           "genres",
           "publishers",
           "esrb_rating",
           }

max_workers = 10
start_game_index = 300000 # min: 0
end_game_index = 300100 # max: 345707
number_of_games = end_game_index - start_game_index
games_per_worker = int(number_of_games/max_workers) 
start_index = range(start_game_index, end_game_index, games_per_worker)

# Make urls
base_url = "https://api.rawg.io/api/games/"
urls = [base_url + csv_data[i][0] for i in range(len(csv_data))]


In [6]:
print(*start_index[:10])
urls[start_game_index:start_game_index+10]

300000 300010 300020 300030 300040 300050 300060 300070 300080 300090


['https://api.rawg.io/api/games/257455',
 'https://api.rawg.io/api/games/257451',
 'https://api.rawg.io/api/games/257449',
 'https://api.rawg.io/api/games/257439',
 'https://api.rawg.io/api/games/257435',
 'https://api.rawg.io/api/games/257433',
 'https://api.rawg.io/api/games/257428',
 'https://api.rawg.io/api/games/257427',
 'https://api.rawg.io/api/games/257426',
 'https://api.rawg.io/api/games/257425']

In [7]:
# Skipped these
downloaded_files = {file.split(".",1)[0] for file in os.listdir("data/game_info/")}

# Time
t0 = time()
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    temp = functools.partial(threading, games_per_worker=games_per_worker, urls=urls, downloaded_files=downloaded_files)
    executor.map(temp, start_index)
    
# Time diff
print(time()-t0)

5.530173301696777
